In [ ]:
import getpass
import os
import json
from langchain_cohere import ChatCohere

#Load the model, langchain has a series of other models
with open(f'api.txt', errors='ignore') as f:
    api_key = f.read()
model = ChatCohere(cohere_api_key=api_key)

#Define conversation id to store multiple 
conversation_id = 1
config = {"configurable": {"thread_id": conversation_id}}

In [ ]:
import sqlite3
conn = sqlite3.connect(":memory:")

db_path = f"memory/id{conversation_id}/memory.db"
conn = sqlite3.connect(db_path, check_same_thread=False)

from langgraph.checkpoint.sqlite import SqliteSaver
memory = SqliteSaver(conn)

In [ ]:
from langchain_core.messages import BaseMessage, SystemMessage, HumanMessage, AIMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """You are a pirate!""",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

In [ ]:
from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, MessagesState, START, END

class State(MessagesState):
    round: str

def call_model(state: State):
    
    
    print("Round: " + state["round"])
    
    #link the initial game rule prompt
    #this can be edited such that it informs the ai with round number but lets not do that for now
    #model can be updated quickly here
    
    chain = prompt | model
    response = chain.invoke({"messages": state["messages"]})
    return {"messages": response}

builder = StateGraph(State)
builder.add_node("call_model", call_model)
builder.add_edge(START, "call_model")
builder.add_edge("call_model", END)

graph = builder.compile(checkpointer=memory)

Try this.

In [ ]:
input_message = {"role": "user", "content": "Yarrr!"}
for chunk in graph.stream(
    {"messages": [input_message]},
    config,
    stream_mode="values",
):
    chunk["messages"][-1].pretty_print()